In [ ]:

### WIP



In [ ]:
# https://github.com/SeldonIO/alibi-detect#drift-detection

In [ ]:
# #pip  install alibi-detect --user
# # # !pip  install tensorflow
# !pip install catboost
# !pip install xgboost
!pip install category_encoders
# # # !pip install torch torchvision torchaudio
# !pip install lightgbm
!pip install verta
!pip install minio


Load List of the required libraries

In [1]:


import alibi

from alibi_detect.cd import TabularDrift,MMDDrift
from alibi_detect.utils.saving import save_detector, load_detector


import pandas as pd
import numpy as np
import matplotlib as mat
import matplotlib.pyplot as plt    
import seaborn as sns
import category_encoders as ce
import joblib
from datetime import datetime

%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from minio import Minio
from verta import Client
# from minio.error import ResponseError

# import pickle
import warnings

In [2]:
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d%Y%H%M%S%f")
experiment_name = "customerchurnuser9"
experiment_id = experiment_name + timestampStr

In [3]:


def get_s3_server():
    minioClient = Minio('minio-ml-workshop:9000',
                    access_key='minio',
                    secret_key='minio123',
                    secure=False)


In [4]:
data = pd.read_csv('../../data/data.csv')
data.head(5)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,148,Male,0,No,No,1,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,45.65,45.65,Yes
1,463,Male,0,Yes,Yes,4,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,No,Electronic check,101.15,385.90,Yes
2,471,Female,1,No,No,17,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,20.65,330.60,No
3,496,Male,0,No,No,22,No,No phone service,DSL,No,...,Yes,No,No,Yes,One year,Yes,Bank transfer (automatic),43.75,903.60,Yes
4,833,Female,0,Yes,Yes,70,Yes,No,DSL,Yes,...,Yes,Yes,No,Yes,One year,No,Credit card (automatic),74.10,5222.30,No


Apply encoding on categorical features

In [5]:
data['Churn'] = data['Churn'].map({'Yes': 1, 'No': 0})

data.replace(" ", np.nan, inplace=True)

data['TotalCharges'] = pd.to_numeric(data['TotalCharges'])

mean = data['TotalCharges'].mean()
data.fillna(mean, inplace=True)
final_set = data.drop(['Churn', 'customerID'], axis=1)
final_set.head(5)
labels = data['Churn']


categories_per_feature = ['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod', 'OnlineSecurity', 'OnlineBackup',
         'DeviceProtection', 'TechSupport']
final_set.head(5)





,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,Male,0,No,No,1,Yes,No,DSL,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,45.65,45.65
1,Male,0,Yes,Yes,4,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,No,Electronic check,101.15,385.90
2,Female,1,No,No,17,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,20.65,330.60
3,Male,0,No,No,22,No,No phone service,DSL,No,Yes,Yes,No,No,Yes,One year,Yes,Bank transfer (automatic),43.75,903.60
4,Female,0,Yes,Yes,70,Yes,No,DSL,Yes,Yes,Yes,Yes,No,Yes,One year,No,Credit card (automatic),74.10,5222.30


In [6]:


labels = data['Churn']
X_train, X_test, Y_train, Y_test = train_test_split(final_set, labels, test_size=0.3)
print ('Training Data Shape',X_train.shape, Y_train.shape)
print ('Testing Data Shape',X_test.shape, Y_test.shape)

Y = data['Churn']
X = final_set
joblib.dump(final_set.columns, 'CustomerChurnDriftFeaturesColumns.pkl')



Training Data Shape (4930, 19) (4930,)
Testing Data Shape (2113, 19) (2113,)


['CustomerChurnDriftFeaturesColumns.pkl']


Define a Drift Model.

Find the categorical columns.

In [7]:
numCols = X_train.select_dtypes("number").columns
catCols = X_train.select_dtypes("object").columns
numCols= list(set(numCols))
catCols= list(set(catCols))
numCols
catCols

['PhoneService',
 'Dependents',
 'Partner',
 'InternetService',
 'PaperlessBilling',
 'PaymentMethod',
 'StreamingTV',
 'OnlineSecurity',
 'MultipleLines',
 'StreamingMovies',
 'Contract',
 'DeviceProtection',
 'OnlineBackup',
 'gender',
 'TechSupport']

In [8]:
numCols

['tenure', 'MonthlyCharges', 'SeniorCitizen', 'TotalCharges']

In [9]:
categories_per_feature = {}
for i in range (len(final_set.columns)):
    print(final_set.columns[i])
    if final_set.columns[i] in catCols:
        categories_per_feature[i] = None
categories_per_feature

gender
SeniorCitizen
Partner
Dependents
tenure
PhoneService
MultipleLines
InternetService
OnlineSecurity
OnlineBackup
DeviceProtection
TechSupport
StreamingTV
StreamingMovies
Contract
PaperlessBilling
PaymentMethod
MonthlyCharges
TotalCharges


{0: None,
 2: None,
 3: None,
 5: None,
 6: None,
 7: None,
 8: None,
 9: None,
 10: None,
 11: None,
 12: None,
 13: None,
 14: None,
 15: None,
 16: None}

In [18]:
save_detector(cd, 'CustomerChurnDriftPredictor')


Directory CustomerChurnDriftPredictor does not exist and is now created.


In [15]:
cd = TabularDrift(X_train.values, p_val=.05, categories_per_feature=categories_per_feature)
joblib.dump(cd, 'CustomerChurnDriftPredictor.sav')
save_detector(cd, 'CustomerChurnDriftPredictor')
# cd1 = load_detector('CustomerChurnDriftPredictor')

['CustomerChurnDriftPredictor.sav']

In [17]:
data_for_check = X_test.copy()
list_dist = []
list_dist_d= []
list_dist_nd= []

list_drift = []
list_drift_index = []
idx =0
cc = 0
for index, row in data_for_check.iterrows():
    cd_preds = cd.predict(data_for_check.iloc[idx].values.reshape(1,data_for_check.shape[1]))
    if cd_preds['data']['is_drift']==1:
        list_drift_index.append(idx)
        list_dist_d.append(np.min(cd_preds['data']['p_val']))

    else:
        list_dist_nd.append(np.min(cd_preds['data']['p_val']))

    list_drift.append(cd_preds['data']['is_drift'])
    list_dist.append(np.mean(cd_preds['data']['p_val']))
    idx +=1
len (list_drift_index)

9

9

In [ ]:
data['Churn'] = data['Churn'].map({'Yes': 1, 'No': 0})

data.replace(" ", np.nan, inplace=True)

data['TotalCharges'] = pd.to_numeric(data['TotalCharges'])

mean = data['TotalCharges'].mean()
data.fillna(mean, inplace=True)


names = ['gender', 'Partner', 'Dependents', 'PhoneService', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling']
# for column in names:
#     labelencoder(column)
data_enc = data
data_enc = data_enc.drop(['Churn', 'customerID'], axis=1)
enc = ce.ordinal.OrdinalEncoder(cols=names)
enc.fit(data_enc)
# ###
# filename_oe = 'OrdinalEncoder_drift.sav'
# pickle.dump(enc, open(filename_oe, 'wb'))

# # some time later...

# # load the model from disk
# enc = pickle.load(open(filename_oe, 'rb'))
# ###
labelled_set = enc.transform(data_enc)

names = ['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod', 'OnlineSecurity', 'OnlineBackup',
         'DeviceProtection', 'TechSupport']

ohe = ce.OneHotEncoder(cols=names)
data_ohe = data
data_ohe = data_ohe.drop(['Churn', 'customerID'], axis=1)
ohe.fit(data_ohe)
# ###
# filename_ohe = 'OneHotEncoder_drift.sav'
# pickle.dump(ohe, open(filename_ohe, 'wb'))

# # some time later...

# # load the model from disk
# ohe = pickle.load(open(filename_ohe, 'rb'))
###
final_set = ohe.transform(labelled_set)
columns_name =final_set.columns
final_set.head(5)

In [ ]:
from sklearn.model_selection import train_test_split
labels = data['Churn']
X_train, X_test, Y_train, Y_test = train_test_split(final_set, labels, test_size=0.3)

Y = data['Churn']
X = final_set

In [ ]:
from tensorflow.keras.layers import Dense, InputLayer, Input,Conv1D,Flatten,Embedding
import tensorflow as tf


n_features = X_train.shape[1]

proj = tf.keras.Sequential(
  [
      InputLayer(input_shape=( n_features,1), name="encode1"),

      Conv1D(32, 4, strides=1, padding='same', activation=tf.nn.relu),
      Conv1D(64, 4, strides=1, padding='same', activation=tf.nn.relu),
      Conv1D(4, 4, strides=1, padding='same', activation=tf.nn.relu),
      Flatten(),
  ]
)

proj.summary()


In [ ]:
from alibi_detect.utils.tensorflow.kernels import DeepKernel
kernel = DeepKernel(proj, eps=0.01)


In [ ]:
# x = X_train.values.reshape(X_train.values.shape[0], X_train.values.shape[1], 1)
# x_t = X_test.values.reshape(X_test.values.shape[0], X_test.values.shape[1], 1).astype('float32')
x = X_train.values.reshape(X_train.values.shape[0],n_features,1).astype('float32')
x_t = X_test.values.reshape(X_test.values.shape[0],n_features,1).astype('float32')
x_t.shape



In [ ]:

from alibi_detect.cd import LearnedKernelDrift
cd = LearnedKernelDrift(x, kernel, backend='tensorflow', p_val=.05, epochs=1)

In [ ]:
x_t0 = x_t[0].reshape(1,n_features,1).astype('float32')

x_t0.shape


In [ ]:
data_for_check = X_test.copy()

cd_preds = cd.predict(x_t0)

In [ ]:
data_for_check = X_test.copy()
list_dist = []
list_dist_d= []
list_dist_nd= []

list_drift = []
list_drift_index = []
idx =0
cc = 0
for index, row in data_for_check.iterrows(): 
    x_t0 = x_t[idx].reshape(1,n_features,1).astype('float32')
    cd_preds = cd.predict(x_t0)
    # cd_preds = cd.predict(data_for_check.iloc[idx].values.reshape(1,data_for_check.shape[1]))
    if cd_preds['data']['is_drift']==1:
        list_drift_index.append(idx)
        list_dist_d.append(np.min(cd_preds['data']['p_val']))
        
    else:
        list_dist_nd.append(np.min(cd_preds['data']['p_val']))

    list_drift.append(cd_preds['data']['is_drift'])
    list_dist.append(np.mean(cd_preds['data']['p_val']))
    idx +=1
len (list_drift_index)